# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

__Описание__

Данные геологоразведки трёх регионов находятся в файлах: 
- /datasets/geo_data_0.csv. 
- /datasets/geo_data_1.csv. 
- /datasets/geo_data_2.csv
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
# импортируем нужные библиотеки

import pandas as pd 
import numpy as np
import warnings
from scipy import stats as st

from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
# отключим некритические предупреждения в коде

warnings.filterwarnings("ignore")

In [3]:
# откроем наши файлы

try:
    data_0 = pd.read_csv('/Users/alex/Downloads/geo_data_0.csv')
    data_1 = pd.read_csv('/Users/alex/Downloads/geo_data_1.csv')
    data_2 = pd.read_csv('/Users/alex/Downloads/geo_data_2.csv')
except:
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
# объявим глобальные переменные

DATA = [data_0, data_1, data_2]

STATE = np.random.RandomState(42)

In [5]:
# переберем в цикле наши дф'ы и выведим основную информацию

for i in range(3):
    print(f'Информация по датасету в датасете {i} : \n\n{DATA[i].head(10)}, \n\n{DATA[i].describe()}')
    print(f'\nКол-во дубликатов в датасете {i} : {DATA[i].duplicated().sum()}')
    print(f'\nКол-во пропусков в датасете {i} : {DATA[i].isna().sum()}')
    print('_' * 70)  
    print('_' * 70)

Информация по датасету в датасете 0 : 

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
5  wX4Hy  0.969570  0.489775 -0.735383   64.741541
6  tL6pL  0.645075  0.530656  1.780266   49.055285
7  BYPU6 -0.400648  0.808337 -5.624670   72.943292
8  j9Oui  0.643105 -0.551583  2.372141  113.356160
9  OLuZU  2.173381  0.563698  9.441852  127.910945, 

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507


Так как __'id'__ - бесполезный признак, удалим его в трех наших датасетах

In [6]:
for i in DATA:
    i.drop(columns = ['id'], axis = 1, inplace = True)
    display(i.head())

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


In [7]:
# взгялнем на матрицу корреляций 

for i in DATA:
    display(i.corr())

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


__Подытожим__, импоритировали нужные нам библиотеки, загружили наши датасеты, проверили нет ли пропусков, а так же удалил ненужные столбец.

В 1 и 3 регионе средние показатели добычи нефти - 91/94 тыс. баррелей, в то время как в регионе 2 - добыча меньше(57 т.бар.) 

У второго региона признак __f2__ сильно коррелирует с целевым признаком.

## Обучение и проверка модели
- Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
- Обучите модель и сделайте предсказания на валидационной выборке.
- Сохраните предсказания и правильные ответы на валидационной выборке.
- Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
- Проанализируйте результаты.

Рассмотрим три региона, обучим модель и сделаем предсказания на валид выборке.

In [8]:
# data_0

# выделим наши фичи и таргет
features_0 = data_0.drop(['product'], axis = 1)
target_0 = data_0['product']

# разделим данные
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(features_0, target_0, 
                                                                              test_size = 0.25, random_state = STATE)

# инициализируем нашу модель, обучим и предскажим
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_0 = model_0.predict(features_valid_0)

# выровним индексы
prediction_valid_0 = pd.Series(predictions_0, index = target_valid_0.index)

# расчитаем среднеквадратичную ошибку(корень)
rmse_0 = mean_squared_error(predictions_0, target_valid_0) ** 0.5

print(f'Среднее значение предсказаний по добыче нефти, для региона 0 : {round(predictions_0.mean(), 2)} тыс.баррелей')
print(f'Средняя фактическая добыча нефти по региону 0 : {round(target_0.mean(), 2)} тыс.баррелей')
print(f'RMSE для региона 0 : {round(rmse_0, 2)}')
print('_' * 70)

Среднее значение предсказаний по добыче нефти, для региона 0 : 92.4 тыс.баррелей
Средняя фактическая добыча нефти по региону 0 : 92.5 тыс.баррелей
RMSE для региона 0 : 37.76
______________________________________________________________________


In [9]:
# data_1

# выделим наши фичи и таргет
features_1 = data_1.drop(['product'], axis = 1)
target_1 = data_1['product']

# разделим данные
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(features_1, target_1, 
                                                                              test_size = 0.25, random_state = STATE)

# инициализируем нашу модель, обучим и предскажим
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_1 = model_1.predict(features_valid_1)

# выровним индексы
prediction_valid_1 = pd.Series(predictions_1, index = target_valid_1.index)

# расчитаем среднеквадратичную ошибку(корень)
rmse_1 = mean_squared_error(predictions_1, target_valid_1) ** 0.5

print(f'Среднее значение предсказаний по добыче нефти, для региона 0 : {round(predictions_1.mean(), 2)} тыс.баррелей')
print(f'Средняя фактическая добыча нефти по региону 0 : {round(target_1.mean(), 2)} тыс.баррелей')
print(f'RMSE для региона 0 : {round(rmse_1, 2)}')
print('_' * 70)

Среднее значение предсказаний по добыче нефти, для региона 0 : 68.59 тыс.баррелей
Средняя фактическая добыча нефти по региону 0 : 68.83 тыс.баррелей
RMSE для региона 0 : 0.89
______________________________________________________________________


In [10]:
# data_2

# выделим наши фичи и таргет
features_2 = data_2.drop(['product'], axis = 1)
target_2 = data_2['product']

# разделим данные
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(features_2, target_2, 
                                                                              test_size = 0.25, random_state = STATE)

# инициализируем нашу модель, обучим и предскажим
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_2 = model_2.predict(features_valid_2)

# выровним индексы
prediction_valid_2 = pd.Series(predictions_2, index = target_valid_2.index)

# расчитаем среднеквадратичную ошибку(корень)
rmse_2 = mean_squared_error(predictions_2, target_valid_2) ** 0.5

print(f'Среднее значение предсказаний по добыче нефти, для региона 0 : {round(predictions_2.mean(), 2)} тыс.баррелей')
print(f'Средняя фактическая добыча нефти по региону 0 : {round(target_2.mean(), 2)} тыс.баррелей')
print(f'RMSE для региона 0 : {round(rmse_2, 2)}')
print('_' * 70)

Среднее значение предсказаний по добыче нефти, для региона 0 : 95.09 тыс.баррелей
Средняя фактическая добыча нефти по региону 0 : 95.0 тыс.баррелей
RMSE для региона 0 : 40.24
______________________________________________________________________


Как можно заметить, для региона __0__ и __2__ , RMSE равно 37/40, что является очень большим значением, лучше всего модель показала себя на регионе __1__, вероятнее всего это из-за сильной зависимости признака __f2__ к __product__

## Подготовка к расчёту прибыли
- Все ключевые значения для расчётов сохраните в отдельных переменных.
- Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе. 
- Напишите выводы по этапу подготовки расчёта прибыли.

In [11]:
# запишем наши ключевые значения

# исследуемые точки
POINTS = 500

# лучшие 200 точек
BEST_POINTS = 200

# бюджет
BUDGET = 10**10

# доход с 1000 баррелей
PRICE = 450000

Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины



In [12]:
min_threshold = BUDGET / (PRICE * BEST_POINTS)
min_threshold

111.11111111111111

In [13]:
# сравним получившийся результат со средним значением запаса в каждом регионе

for i in range(3):
    print(f'Минимальный объем сырья для безубыточной разработки: {round(min_threshold, 2)} тыс.баррелей')
    print(f'\nСредний запас сырья в регионе {i}: {round(DATA[i]["product"].mean(), 2)} тыс.баррелей')
    print(f'Кол-во объектов, имеющих запас сырья больше чем мин.объем: {len(DATA[i].query("product > @min_threshold"))}')
    print('_' * 70)
    print('_' * 70)

Минимальный объем сырья для безубыточной разработки: 111.11 тыс.баррелей

Средний запас сырья в регионе 0: 92.5 тыс.баррелей
Кол-во объектов, имеющих запас сырья больше чем мин.объем: 36583
______________________________________________________________________
______________________________________________________________________
Минимальный объем сырья для безубыточной разработки: 111.11 тыс.баррелей

Средний запас сырья в регионе 1: 68.83 тыс.баррелей
Кол-во объектов, имеющих запас сырья больше чем мин.объем: 16537
______________________________________________________________________
______________________________________________________________________
Минимальный объем сырья для безубыточной разработки: 111.11 тыс.баррелей

Средний запас сырья в регионе 2: 95.0 тыс.баррелей
Кол-во объектов, имеющих запас сырья больше чем мин.объем: 38178
______________________________________________________________________
______________________________________________________________________


Сравнивая полученный объём сырья со средним запасом сырья в каждом регионе - можно сказать следующее, что средние запасы в каждом из регионов значительно ниже минимального объема, однако если смотреть в разрезе то в каждом регионе есть объекты - хранящие в себе больше сырья чем безубыточный порог.

## Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [14]:
# функция для посчета прибыли на основе предсказаний, цены и кол-ва скважин:

def revenue(target, probabilities, count, price): 
    # сортируем предсказания по убыванию
    probs_sorted = probabilities.sort_values(ascending = False) 
    # выбираем скважины с макс. значениями предсказаний до нужного кол-ва
    selected = target[probs_sorted.index][:count]  
    # возвращаем прибыль (доход с 1000 бар. * скважины - бюджет на разработку)
    return price * selected.sum() - BUDGET


## Посчитайте риски и прибыль для каждого региона:

- Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
- Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
- Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

Напишем функцию bootstrap и применим ее на 3 наших регионах

In [15]:
def bootstrap(target, valid, points, state):
    values = []
    for i in range(1000):
        # выберем 500 скважин
        target_subsample = target.sample(n = points, replace = True, random_state = state)
        # берем соответсвующие предсказания для этих скважин
        probs_subsample = valid[target_subsample.index]
        # применяем нашу фунцкцию по расцету прибыли
        rev = revenue(target_subsample, probs_subsample, BEST_POINTS, PRICE)
        values.append(rev)
        
    values = pd.Series(values)
    return values

In [16]:
# для data_0

values_0 = bootstrap(target_valid_0, prediction_valid_0, POINTS, STATE)

mean_0 = values_0.mean()
lower_0 = values_0.quantile(0.025)
upper_0 = values_0.quantile(0.975)
risk_0 = values_0[values_0 < 0].count() / values_0.count() * 100

print(f'Средняя прибыль : {round(mean_0, 2)}')
print(f'Доверительный интервал (95%) : от {round(lower_0, 2)} до {round(upper_0, 2)}')
print(f'Риск получения убытков: {round(risk_0, 2)}%')

Средняя прибыль : 440446619.54
Доверительный интервал (95%) : от -129698263.88 до 980351534.78
Риск получения убытков: 5.8%


In [17]:
# для data_1

values_1 = bootstrap(target_valid_1, prediction_valid_1, POINTS, STATE)

mean_1 = values_1.mean()
lower_1 = values_1.quantile(0.025)
upper_1 = values_1.quantile(0.975)
risk_1 = values_1[values_1 < 0].count() / values_1.count() * 100

print(f'Средняя прибыль : {round(mean_1, 2)}')
print(f'Доверительный интервал (95%) : от {round(lower_1, 2)} до {round(upper_1, 2)}')
print(f'Риск получения убытков: {round(risk_1, 2)}%')

Средняя прибыль : 489036146.09
Доверительный интервал (95%) : от 63878578.77 до 900130455.53
Риск получения убытков: 1.0%


In [18]:
# для data_2

values_2 = bootstrap(target_valid_2, prediction_valid_2, POINTS, STATE)

mean_2 = values_2.mean()
lower_2 = values_2.quantile(0.025)
upper_2 = values_2.quantile(0.975)
risk_2 = values_2[values_2 < 0].count() / values_2.count() * 100

print(f'Средняя прибыль : {round(mean_2, 2)}')
print(f'Доверительный интервал (95%) : от {round(lower_2, 2)} до {round(upper_2, 2)}')
print(f'Риск получения убытков: {round(risk_2, 2)}%')

Средняя прибыль : 343559871.28
Доверительный интервал (95%) : от -210907750.68 до 884771214.58
Риск получения убытков: 11.9%


# Вывод

По результатам исследования можно предложить следующее, __наиболее выгодное место для разработки скважин - является  второе месторождение data_1(geo_data_1).__ Т.к данный регион имеет бОльшую __среднюю прибыль, меньший % риска получить отрицательную прибыль (единственный имеет положительную "левую границу" 95%го доверительного интервала)__ 